In [58]:
from fyers_apiv3 import fyersModel
from fyers_api import accessToken
import pandas as pd
import os
import numpy as np
client_id = "TNRLEAR9UP-100"
is_token = False
content = ""
if os.path.exists("access_token.txt"):
    with open("access_token.txt","r") as fp:
        access_token = fp.read()
        print("Access token is available in .txt File")
        if access_token:
            is_token = True
if is_token==False:
    print("Token not available")
    from fyers_api import accessToken,fyersModel
    client_id = "TNRLEAR9UP-100"
    secret_key = "VCFFWC2WIA"
    redirect_uri = "https://www.elevenunit.com/"
    response_type = "code"
    state = "sample_state"
    grant_type="authorization_code"
    
    session=accessToken.SessionModel(
        client_id=client_id,
        secret_key=secret_key,
        redirect_uri=redirect_uri, 
        response_type=response_type,
        grant_type=grant_type
    )
    
    response = session.generate_authcode()
    print("Login URL::",response)
    
    auth_code = input("Enter auth code")
    session.set_token(auth_code)
    print(session)
    
    response = session.generate_token()
    print(response)

    
    access_token = response['access_token']
    with open("access_token.txt","w") as f:
        f.write(access_token)


Access token is available in .txt File


In [32]:
# fyers = fyersModel.FyersModel(client_id=client_id, token=access_token)
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

In [162]:
from datetime import datetime,timedelta
### present date
today_timestamp = datetime.now()
curr_date = today_timestamp.strftime("%Y-%m-%d")

#previous date
prev_date = today_timestamp - timedelta(days = 60)
print(type(prev_date),prev_date)
str_prev_date = prev_date.strftime("%Y-%m-%d")
curr_date = today_timestamp - timedelta(days=0)
str_curr_date = curr_date.strftime("%Y-%m-%d")
print("Start Date:",str_prev_date)
print("End Date  :",str_curr_date)
print("No. of days :",(curr_date - prev_date).days)

<class 'datetime.datetime'> 2023-10-11 23:02:04.885818
Start Date: 2023-10-11
End Date  : 2023-12-10
No. of days : 60


In [163]:
# https://api-docs.fyers.in/docs/index.html#tag/Data-Api/paths/~1DataApi/post
symbol="NSE:SBIN-EQ"
# symbol="NSE:NIFTY50-INDEX"


In [164]:

symbol="NSE:SBIN-EQ"
data = {"symbol":symbol,"resolution":"60","date_format":"1","range_from":str_prev_date,"range_to":str_curr_date,"cont_flag":"1"}
data = fyers.history(data)

if data['s']!='ok':
    print("Error is accessing data",data)
else:
    print("Congratulation!! you got the data")

Congratulation!! you got the data


In [165]:
df = pd.DataFrame(data['candles'],columns=['Date','Open','High','Low','Close','Volume'])
df['Date']=df['Date'].apply(lambda x:datetime.fromtimestamp(x))
df

,Date,Open,High,Low,Close,Volume
0,2023-10-11 09:15:00,596.90,597.00,594.00,594.95,2213195
1,2023-10-11 10:15:00,594.70,595.25,591.95,592.65,1519481
2,2023-10-11 11:15:00,592.60,595.30,592.25,594.80,1124544
3,2023-10-11 12:15:00,594.80,595.00,592.60,593.60,969286
4,2023-10-11 13:15:00,593.55,593.60,588.65,589.00,2489295
...,...,...,...,...,...,...
276,2023-12-08 11:15:00,611.45,612.60,609.50,609.80,1148870
277,2023-12-08 12:15:00,609.80,610.00,605.95,607.90,2361319
278,2023-12-08 13:15:00,607.90,609.95,605.35,609.05,1932298
279,2023-12-08 14:15:00,609.00,614.50,608.10,613.85,3504833


In [166]:
df['Ema9']= df['Close'].ewm(span=9, adjust=False).mean().round(2)
df['Ema15']= df['Close'].ewm(span=15, adjust=False).mean().round(2)
diff = 0
df['EmaStatus'] = np.where(df['Ema9'] + diff > df['Ema15'],'UP',np.where(df['Ema9'] + diff < df['Ema15'],'DOWN','RANGE'))
df

,Date,Open,High,Low,Close,Volume,Ema9,Ema15,EmaStatus
0,2023-10-11 09:15:00,596.90,597.00,594.00,594.95,2213195,594.95,594.95,RANGE
1,2023-10-11 10:15:00,594.70,595.25,591.95,592.65,1519481,594.49,594.66,DOWN
2,2023-10-11 11:15:00,592.60,595.30,592.25,594.80,1124544,594.55,594.68,DOWN
3,2023-10-11 12:15:00,594.80,595.00,592.60,593.60,969286,594.36,594.54,DOWN
4,2023-10-11 13:15:00,593.55,593.60,588.65,589.00,2489295,593.29,593.85,DOWN
...,...,...,...,...,...,...,...,...,...
276,2023-12-08 11:15:00,611.45,612.60,609.50,609.80,1148870,610.22,608.95,UP
277,2023-12-08 12:15:00,609.80,610.00,605.95,607.90,2361319,609.75,608.82,UP
278,2023-12-08 13:15:00,607.90,609.95,605.35,609.05,1932298,609.61,608.85,UP
279,2023-12-08 14:15:00,609.00,614.50,608.10,613.85,3504833,610.46,609.47,UP


In [173]:
# df0 = df.head(60)
# print(df0.shape[0])
status = list()
diff_list = list()
info = list()
in_market = False
for index,row in df.iterrows():
    if index==0 or index == 1:
        # prev_close = row.Close
        # prev_ema9 = row.Ema9
        # prev_ema15 = row.Ema15
        # last_diff = bool(prev_ema9 - prev_ema15)
        # print(type(df.loc[0,['Ema9']].values),df.loc[0,['Ema9']].values)
        last_diff = df.loc[index,['Ema9']].values[0] - df.loc[index,['Ema15']].values[0] 
        diff_list.append(last_diff)
        status.append('None')
        info.append('None')
        continue
    else:
        curr_diff = row.Ema9 - row.Ema15
        diff_list.append(curr_diff)
        info.append([last_diff,curr_diff])
        if (bool(last_diff>0) and bool(curr_diff>0)) or  (bool(last_diff<0) and bool(curr_diff<0)) or (row.Ema9 == row.Ema15):
            if in_market:
                status.append('HOLD')
            else:
                status.append('None')
        else:
            in_market = True
            if last_diff>0 and curr_diff<0:
                print(status[-1])
                if status[-1] =='HOLD' or status[-1] =='None':
                    status.append('PE')
                else:
                    status.append('HOLD')
            else:
                if status[-1] =='HOLD' or status[-1] =='None':
                    status.append('CE')
                else:
                    status.append('HOLD')
                
            
    last_diff = df.loc[index-1,['Ema9']].values[0] - df.loc[index-1,['Ema15']].values[0] 
    # last_diff = df.loc[index,['Ema9']].values[0] - df.loc[index,['Ema15']].values[0]
    # diff_list.append(last_diff)
    # print(index,status[-1])
df['Call']=status
df['Diff']=diff_list
df['info']=info
# print("::",len(status))
# print("::",len(diff_list))
display(df.iloc[-130:-80,:])
# display(df.tail(65))
df.to_csv("report.csv")

HOLD
PE
HOLD
HOLD
PE


,Date,Open,High,Low,Close,Volume,Ema9,Ema15,EmaStatus,Call,Diff,info
151,2023-11-10 13:15:00,579.10,579.95,578.50,579.35,1130283,578.69,578.60,UP,HOLD,0.09,"[-0.029999999999972715, 0.09000000000003183]"
152,2023-11-10 14:15:00,579.25,579.80,578.05,579.30,1440638,578.81,578.69,UP,HOLD,0.12,"[0.03999999999996362, 0.11999999999989086]"
153,2023-11-10 15:15:00,579.30,580.90,579.00,580.25,575095,579.10,578.88,UP,HOLD,0.22,"[0.09000000000003183, 0.22000000000002728]"
154,2023-11-12 18:15:00,584.60,584.60,580.30,580.80,1944795,579.44,579.12,UP,HOLD,0.32,"[0.11999999999989086, 0.32000000000005]"
155,2023-11-13 09:15:00,581.00,581.00,577.00,577.05,1390013,578.96,578.86,UP,HOLD,0.10,"[0.22000000000002728, 0.10000000000002274]"
156,2023-11-13 10:15:00,577.15,577.50,575.20,577.00,2615282,578.57,578.63,DOWN,PE,-0.06,"[0.32000000000005, -0.05999999999994543]"
157,2023-11-13 11:15:00,577.00,579.60,576.65,579.30,1532747,578.72,578.71,UP,HOLD,0.01,"[0.10000000000002274, 0.009999999999990905]"
158,2023-11-13 12:15:00,579.30,579.55,578.10,578.75,1537482,578.72,578.72,RANGE,HOLD,0.00,"[-0.05999999999994543, 0.0]"
159,2023-11-13 13:15:00,578.85,579.40,578.25,579.25,1206096,578.83,578.79,UP,HOLD,0.04,"[0.009999999999990905, 0.04000000000007731]"
160,2023-11-13 14:15:00,579.20,582.50,578.50,581.30,2233993,579.32,579.10,UP,CE,0.22,"[0.0, 0.22000000000002728]"
